In [150]:
from classiq import (
    Model,
    RegisterUserInput,
    FunctionLibrary,
    FunctionGenerator,
    RegisterUserInput,
    QASM_INTRO,
    qfunc,
    QReg,
    QUInt,
)
from classiq.builtin_functions import ArithmeticOracle, GroverOperator, CustomOracle, StatePreparation
from classiq import execute, show, synthesize
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq.interface.generator.state_preparation import ComputationalBasisStatePreparation
from classiq.synthesis import set_execution_preferences

In [151]:
# probability_free_cells = [1, 0, 0, 0]
# probability_clue = [0, 0, 0, 1]
#
# clue_preparation_params = StatePreparation(probabilities=probability_clue)
# free_preparation_params = StatePreparation(probabilities=probability_free_cells)

# register_references = [m.StatePreparation(params=clue_preparation_params)]
# for i in range(1, 9):
#     register_references.append(m.StatePreparation(params=free_preparation_params))

In [159]:
@qfunc
def my_state_preparation_1(args: QReg[18]) -> QReg[18]:
    state = "qreg q[18];\n"
    # prepare current node
    state += f"h q[3];\n"
    # prepare next node
    state += "h q[4];\nh q[5];"
    return QASM_INTRO + state

@qfunc
def my_state_preparation_2(args_2: QReg[18]) -> QReg[18]:
    state = "qreg q[18];\n"
    # prepare current node
    state += f"h q[3];\n"
    # prepare next node
    state += "h q[6];\nh q[7];"
    return QASM_INTRO + state


@qfunc
def my_oracle(args: QReg[18]) -> QReg[18]:
    return QASM_INTRO + "qreg q[18];"

my_function_library = FunctionLibrary(my_state_preparation_1, my_oracle, my_state_preparation_2)

In [160]:
new_oracle_params = CustomOracle(
    custom_oracle= "my_oracle",
    custom_oracle_params= my_function_library.get_function("my_oracle"),
)

In [161]:
grover_params_1 = GroverOperator(
    args=QReg[18],
    oracle_params=new_oracle_params,
    state_preparation="my_state_preparation_1",
    state_preparation_params=my_function_library.get_function("my_state_preparation_1"),
)

grover_params_2 = GroverOperator(
    args=QReg[18],
    oracle_params=new_oracle_params,
    state_preparation="my_state_preparation_2",
    state_preparation_params=my_function_library.get_function("my_state_preparation_2"),
)

In [162]:
model = Model()
model.include_library(my_function_library)

In [164]:
initial_state = "110000000000000000" # First cell is 3, everything else is 0

# prepare the board
clue_preparation_params = ComputationalBasisStatePreparation(computational_state=initial_state)
prepared_board = model.ComputationalBasisStatePreparation(params=clue_preparation_params)

# prepare the 2 diffusion operators
grover_1 = model.GroverOperator(grover_params_1, in_wires={"args": prepared_board['OUT']})
grover_2 = model.GroverOperator(grover_params_2, in_wires={"args": grover_1['args']})

In [165]:
quantum_program = synthesize(model.get_model())
show(quantum_program)

Opening: https://platform.classiq.io/circuit/e74a6f97-e338-436c-bf8d-a0bbe0820044?version=0.34.0
